In [41]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [42]:
import sys
LOCAL_FAST_AI = '/Users/krishnakalyan3/Educational/FastAI/fastai/'
AWS_FAST_AI = '/home/ubuntu/fastai'
sys.path.append(AWS_FAST_AI)

In [43]:
from fastai.transforms import * 
from fastai.conv_learner import * 
from fastai.model import *
from fastai.dataset import * 
from fastai.sgdr import *
from fastai.plots import *

In [44]:
PATH = '/home/ubuntu/Plant/data_all/'

In [45]:
label_csv = f'{PATH}all.csv'
n = len(list(open(label_csv)))-1
val_idxs = get_cv_idxs(n, val_pct=.0004)
n, len(val_idxs)

(5544, 2)

In [46]:
f_model = resnet50
sz=250
tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_top_down, max_zoom=1.05)

In [47]:
data = ImageClassifierData.from_csv(PATH, folder='all_train', csv_fname=f'{PATH}all.csv',
                                    tfms=tfms, val_idxs=val_idxs, test_name='test',  bs=64)

In [ ]:
learn = ConvLearner.pretrained(f_model, data, precompute=True)
learn.models.model = torch.nn.DataParallel(learn.models.model,device_ids=list(range(0,8)))

 75%|███████▍  | 65/87 [00:44<00:14,  1.47it/s]

In [32]:
learn.precompute=False
lr = 1e-1
learn.fit(lr, 3, cycle_len=1)

In [ ]:
learn.unfreeze()
lr=np.array([lr/18,lr/6,lr/2])
learn.fit(lr, 3, cycle_len=2, cycle_mult=4)

In [ ]:
learn.save(f'resnet_50_all_{sz}')